# Preprocessing

### Settings

In [13]:
import random
import os
from os import path

from PIL import Image

from data_management import data_utils
from data_management.image_manipulations import image_to_dataloader_folders, crop_bottom_and_sides

seed = 1

In [14]:
root_dir = '/media/alex/A4A034E0A034BB1E/incidents-thesis/geograph-coords-data/'
output_dir = path.join(root_dir, 'incidents_cleaned')

split_probabilities = {'train':70, 'val':20, 'test':10}

# Positives

In [15]:
split_field_name = 'split'
database = '/media/alex/A4A034E0A034BB1E/incidents-thesis/correctimgs'

**Geograph positives**

In [4]:
# image_table_name = 'incidents_geograph'
# handler = data_utils.ImgDatabaseHandler(database)
# geograph_root = '/media/alex/A4A034E0A034BB1E/Geograph'

# try:
#     handler.add_field(image_table_name, split_field_name) # Only once - note that SQLite does not allow deletes
# except Exception as e:
#     print(e)

# handler.calculate_splits(image_table_name, split_field_name, split_probabilities)
# records = handler.get_all_records(image_table_name)

# all_recorded_classes = []
# [all_recorded_classes.append(str(record[4])) for record in records]
# unique_classes = set(all_recorded_classes)
# data_utils.create_dataloader_folders(root_dir, output_dir, unique_classes)

# for record in records:
#     filepath = os.path.join(geograph_root, record[0])
#     # dataloader_root, img_class, img_split, img_path
#     image_to_dataloader_folders(output_dir, record[4], record[-1], filepath, output_img_width=500)

# handler.db.close()

duplicate column name: split


## Negatives

**Geograph negatives:** 9981 photos

In [16]:
negatives_class = ['negative']
data_utils.create_dataloader_folders(root_dir, output_dir, negatives_class)

In [26]:
seed = 1
root_dir = '/media/alex/A4A034E0A034BB1E/incidents-thesis/geograph-coords-data/'
output_dir = path.join(root_dir, 'incidents_cleaned')    
negatives_class = ['negative']
data_utils.create_dataloader_folders(root_dir, output_dir, negatives_class)

split_probabilities = {'train':0.7, 'val':0.2, 'test':0.1}

geograph_root = '/media/alex/A4A034E0A034BB1E/incidents-thesis/true-negatives/geograph/road-incidents-all/'
geograph_negatives = path.join(geograph_root, 'geophotos')
geograph_metadata_file = path.join(geograph_root, 'road-incidents-all.csv')

images = data_utils.sample_n_images_from_dir(geograph_negatives, seed, 10000)
imgs_from_root = [img.split(geograph_root)[-1] for img in images]
imgs_with_coords = data_utils.get_geograph_coords(imgs_from_root, geograph_metadata_file)
imgs_in_split = data_utils.get_coord_splits(imgs_with_coords, split_probabilities)

train_imgs = imgs_in_split['train']
val_imgs = imgs_in_split['val']
test_imgs = imgs_in_split['test']

splits = ['train','val','test']
for i, imgs in enumerate([train_imgs, val_imgs, test_imgs]):
    for index, image in enumerate(imgs):
        target_img = path.join(geograph_root, image[-1])
        image_to_dataloader_folders(output_dir, 'negative', splits[i], target_img, output_img_width=500)
        if (index+1) % 1000 == 0:
            print(splits[i], ': ', index)
            print(splits[i], ': ', image[5]) # Latitude increments

Not enough images in the set, retaining all (9981)
train :  999
train :  0.894902
train :  1999
train :  0.905017
train :  2999
train :  0.912865
train :  3999
train :  0.921299
train :  4999
train :  0.931576
train :  5999
train :  0.942133
val :  999
val :  0.963000


### Replace table in case of faulty splits

https://stackoverflow.com/questions/8442147/how-to-delete-or-add-column-in-sqlite

CREATE TABLE incidents_geo (
 img_id VARCHAR PRIMARY KEY, 
 lat REAL,
 long REAL,
 datetime VARCHAR,
 class STRING
);

INSERT INTO incidents_geo
SELECT img_id, lat, long, datetime, class FROM incidents_geograph;